In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [ ]:
%run -p ../train_cnf_disentangle_rl.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/cnf_cifar10_bs900_rl_stdscale_15_annealing_run3 --seed 3 --lr 0.001 --conditional False --controlled_tol False --log_freq 10 --scale_fac 1.0 --scale_std 15.0 --annealing_std True
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
GATES = ["cnn1", "cnn2", "rnn"]

parser = argparse.ArgumentParser("Continuo

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0


/tancode/repos/tan-ffjord/lib/layers/odefunc_rl.py:291: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iter 0010 | Time 11.6831(31.3754) | Bit/dim 8.7878(8.9815) | Xent 0.0000(0.0000) | Loss 18.4827(18.5225) | Error 0.0000(0.0000) Steps 472(434.42) | Grad Norm 18.0980(27.3404) | Total Time 0.00(0.00)
Iter 0020 | Time 11.4106(26.1879) | Bit/dim 8.5197(8.8913) | Xent 0.0000(0.0000) | Loss 17.6605(18.3608) | Error 0.0000(0.0000) Steps 430(435.71) | Grad Norm 8.5127(23.0497) | Total Time 0.00(0.00)
Iter 0030 | Time 10.6053(22.2983) | Bit/dim 8

...

Iter 6770 | Time 20.7761(20.0937) | Bit/dim 3.4637(3.4701) | Xent 0.0000(0.0000) | Loss 8.1532(9.0418) | Error 0.0000(0.0000) Steps 700(731.74) | Grad Norm 3.9913(5.1453) | Total Time 0.00(0.00)
Iter 6780 | Time 19.5032(20.0903) | Bit/dim 3.4421(3.4679) | Xent 0.0000(0.0000) | Loss 8.2458(8.8329) | Error 0.0000(0.0000) Steps 724(734.19) | Grad Norm 3.1082(5.2809) | Total Time 0.00(0.00)
Iter 6790 | Time 19.5995(20.0737) | Bit/dim 3.4327(3.4685) | Xent 0.0000(0.0000) | Loss 8.1598(8.6634) | Error 0.0000(0.0000) Steps 718(734.41) | Grad Norm 4.1285(4.9773) | Total Time 0.00(0.00)
Iter 6800 | Time 20.5550(20.0269) | Bit/dim 3.4711(3.4688) | Xent 0.0000(0.0000) | Loss 8.2551(8.5441) | Error 0.0000(0.0000) Steps 742(734.58) | Grad Norm 6.4714(5.2655) | Total Time 0.00(0.00)
Iter 6810 | Time 20.2898(19.9493) | Bit/dim 3.4579(3.4670) | Xent 0.0000(0.0000) | Loss 8.2770(8.4516) | Error 0.0000(0.0000) Steps 730(731.54) | Grad Norm 6.3307(5.4797) | Total Time 0.00(0.00)
Iter 6820 | Time 20.2168(

11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006


Iter 6830 | Time 20.0617(20.1326) | Bit/dim 3.4584(3.4705) | Xent 0.0000(0.0000) | Loss 8.1202(8.9448) | Error 0.0000(0.0000) Steps 754(735.50) | Grad Norm 3.9048(5.0259) | Total Time 0.00(0.00)
Iter 6840 | Time 20.2377(20.1279) | Bit/dim 3.5092(3.4699) | Xent 0.0000(0.0000) | Loss 8.4428(8.7690) | Error 0.0000(0.0000) Steps 748(739.19) | Grad Norm 12.3032(5.2603) | Total Time 0.00(0.00)
Iter 6850 | Time 20.9228(20.1626) | Bit/dim 3.4574(3.4686) | Xent 0.0000(0.0000) | Loss 8.1281(8.6202) | Error 0.0000(0.0000) Steps 700(736.13) | Grad Norm 6.1229(5.4900) | Total Time 0.00(0.00)
Iter 6860 | Time 20.1301(20.1393) | Bit/dim 3.4335(3.4682) | Xent 0.0000(0.0000) | Loss 8.1598(8.5234) | Error 0.0000(0.0000) Steps 748(739.42) | Grad Norm 3.4719(5.3449) | Total Time 0.00(0.00)
Iter 6870 | Time 20.0233(20.0580) | Bit/dim 3.5129(3.4675) | Xent 0.0000(0.0000) | Loss 8.3873(8.4450) | Error 0.0000(0.0000) Steps 742(740.38) | Grad Norm 5.2085(5.1892) | Total Time 0.00(0.00)
validating...
Epoch 0125

11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012


===> Using batch size 900. Total 55 iterations/epoch.
Iter 6880 | Time 20.7362(20.0995) | Bit/dim 3.4443(3.4682) | Xent 0.0000(0.0000) | Loss 8.0644(9.0587) | Error 0.0000(0.0000) Steps 700(738.35) | Grad Norm 4.2917(5.4157) | Total Time 0.00(0.00)
Iter 6890 | Time 20.9745(20.1555) | Bit/dim 3.4462(3.4647) | Xent 0.0000(0.0000) | Loss 8.2164(8.8320) | Error 0.0000(0.0000) Steps 790(739.16) | Grad Norm 5.6213(5.4699) | Total Time 0.00(0.00)
Iter 6900 | Time 21.5405(20.2681) | Bit/dim 3.4403(3.4649) | Xent 0.0000(0.0000) | Loss 8.0631(8.6738) | Error 0.0000(0.0000) Steps 670(738.18) | Grad Norm 7.5352(5.2138) | Total Time 0.00(0.00)
